In [1]:
from faker import Faker
from PIL import Image, ImageDraw, ImageFont
import random
import os
import csv
import json
import shutil
from tqdm import tqdm

# Configuración inicial
random.seed(42)  # Para reproducibilidad
fake = Faker('es_MX')

In [2]:
# Configuración de carpetas
BASE_DIR = r"C:\Users\Adrian\Desktop\INEs Dataset"  # Directorio base
DATA_DIR = os.path.join(BASE_DIR, "data")  # Carpeta para guardar todas las imágenes
SETS_DIR = os.path.join(BASE_DIR, "sets")  # Carpeta para los conjuntos
TRAIN_DIR = os.path.join(SETS_DIR, "train")  # Carpeta de entrenamiento
VAL_DIR = os.path.join(SETS_DIR, "validation")     # Carpeta de validación
TEST_DIR = os.path.join(SETS_DIR, "test")    # Carpeta de prueba

In [3]:
# Crear estructura de carpetas
def crear_estructura_carpetas():
    """
    Crea la estructura de carpetas necesaria para el proyecto.
    """
    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(TRAIN_DIR, exist_ok=True)
    os.makedirs(VAL_DIR, exist_ok=True)
    os.makedirs(TEST_DIR, exist_ok=True)
    print("Estructura de carpetas creada correctamente.")

In [4]:
# Función para eliminar acentos
def eliminar_acentos(texto):
    """
    Elimina los acentos y caracteres especiales de un texto.
    
    Args:
        texto (str): Texto con posibles acentos.
        
    Returns:
        str: Texto sin acentos.
    """
    reemplazos = {
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Ü': 'U', 'ü': 'u', 'Ñ': 'N', 'ñ': 'n'
    }
    for acento, sin_acento in reemplazos.items():
        texto = texto.replace(acento, sin_acento)
    return texto

In [5]:
# Función para generar un CURP válido en formato
def generar_curp():
    """
    Genera un CURP aleatorio con formato válido.
    
    Returns:
        str: Un CURP aleatorio.
    """
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato básico de CURP mexicano (simplificado)
    curp = ""
    curp += ''.join(random.choices(letras, k=4))  # Primeras 4 letras (apellidos y nombre)
    curp += ''.join(random.choices(numeros, k=6))  # Fecha (AAMMDD)
    curp += random.choice("HM")  # Sexo
    curp += ''.join(random.choices(letras, k=2))  # Estado
    curp += ''.join(random.choices(letras, k=3))  # Consonantes internas
    curp += random.choice(numeros + letras)  # Homoclave
    curp += ''.join(random.choices(numeros, k=1))  # Dígito verificador
    
    return curp

In [6]:
# Función para generar un número de elector válido
def generar_clave_elector():
    """
    Genera una clave de elector aleatoria con formato válido.
    
    Returns:
        str: Una clave de elector aleatoria.
    """
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato típico de clave de elector
    clave = ""
    clave += ''.join(random.choices(letras, k=6))  # Primeras letras (apellidos y nombre)
    clave += ''.join(random.choices(numeros, k=8))  # Fecha y código
    clave += ''.join(random.choices(letras + numeros, k=4))  # Homoclave
    
    return clave

In [7]:
# Función para generar abreviación de estado
def abreviar_estado(estado):
    """
    Convierte el nombre de un estado mexicano a su abreviatura.
    
    Args:
        estado (str): Nombre completo del estado.
        
    Returns:
        str: Abreviatura del estado.
    """
    # Abreviaturas de estados mexicanos
    abreviaturas = {
        "AGUASCALIENTES": "AGS",
        "BAJA CALIFORNIA": "BC",
        "BAJA CALIFORNIA SUR": "BCS",
        "CAMPECHE": "CAMP",
        "CHIAPAS": "CHIS",
        "CHIHUAHUA": "CHIH",
        "COAHUILA": "COAH",
        "COLIMA": "COL",
        "DISTRITO FEDERAL": "CDMX",
        "CIUDAD DE MEXICO": "CDMX",
        "DURANGO": "DGO",
        "GUANAJUATO": "GTO",
        "GUERRERO": "GRO",
        "HIDALGO": "HGO",
        "JALISCO": "JAL",
        "MEXICO": "MEX",
        "ESTADO DE MEXICO": "MEX",
        "MICHOACAN": "MICH",
        "MORELOS": "MOR",
        "NAYARIT": "NAY",
        "NUEVO LEON": "NL",
        "OAXACA": "OAX",
        "PUEBLA": "PUE",
        "QUERETARO": "QRO",
        "QUINTANA ROO": "Q.ROO",
        "SAN LUIS POTOSI": "SLP",
        "SINALOA": "SIN",
        "SONORA": "SON",
        "TABASCO": "TAB",
        "TAMAULIPAS": "TAMPS",
        "TLAXCALA": "TLAX",
        "VERACRUZ": "VER",
        "YUCATAN": "YUC",
        "ZACATECAS": "ZAC"
    }
    
    # Si el estado está en la lista, usar su abreviatura, de lo contrario abreviar las primeras 3 letras
    estado_limpio = eliminar_acentos(estado.upper())
    if estado_limpio in abreviaturas:
        return abreviaturas[estado_limpio]
    else:
        return estado_limpio[:3]

In [8]:
def generar_direccion_completa():
    """
    Genera una dirección completa aleatoria.
    
    Returns:
        dict: Diccionario con los componentes de la dirección.
    """
    # Generar calle sin acentos
    calle = eliminar_acentos(fake.street_name().upper())
    
    # Generar número exterior con variedad de formatos
    formatos_ext = [
        str(random.randint(1, 9999)),  # Número simple: 123
        str(random.randint(1, 9999)) + "-" + str(random.randint(1, 99)),  # Con guión: 123-45
        str(random.randint(1, 9999)) + random.choice("ABCDEFGHIJ"),  # Con letra: 123A
        "S/N",  # Sin número
        "MZ " + str(random.randint(1, 99)) + " LT " + str(random.randint(1, 99)),  # Formato manzana-lote
        "KM " + str(random.randint(1, 100)) + "." + str(random.randint(0, 9))  # Kilómetro carretera
    ]
    num_ext = random.choice(formatos_ext)
    
    # Decidir si tiene número interior (30% de probabilidad)
    tiene_interior = random.random() < 0.3
    
    # Generar número interior con variedad
    num_int = ""
    if tiene_interior:
        formatos_int = [
            "INT " + str(random.randint(1, 999)),  # INT 123
            "DEPTO " + str(random.randint(1, 999)),  # DEPTO 123
            "DEPTO " + random.choice("ABCDEFGHIJ"),  # DEPTO A
            random.choice("ABCDEFGHIJ"),  # Sólo letra: A
            "PISO " + str(random.randint(1, 20)),  # PISO 3
            "EDIFICIO " + str(random.randint(1, 20)) + " DEPTO " + str(random.randint(1, 99))  # EDIFICIO 3 DEPTO 12
        ]
        num_int = random.choice(formatos_int)
    
    # Generar el resto de componentes
    colonia = eliminar_acentos(fake.city_suffix().upper() + " " + fake.city_prefix().upper())
    cp = str(random.randint(10000, 99999))  # Códigos postales en México son de 5 dígitos
    municipio = eliminar_acentos(fake.city().upper())
    estado = eliminar_acentos(fake.state().upper())
    estado_abreviado = abreviar_estado(estado)
    
    # Formatear la dirección completa
    direccion_linea1 = f"{calle} {num_ext}"
    if tiene_interior:
        direccion_linea1 += f", {num_int}"
        
    direccion_linea2 = f"{colonia} {cp}"
    direccion_linea3 = f"{municipio}, {estado_abreviado}"
    
    # Retornar componentes individuales y líneas formateadas
    return {
        "direccion1": direccion_linea1,
        "direccion2": direccion_linea2,
        "direccion3": direccion_linea3,
        "calle": calle,
        "num_ext": num_ext,
        "num_int": num_int,
        "colonia": colonia,
        "cp": cp,
        "municipio": municipio,
        "estado": estado_abreviado
    }

In [9]:
# Función para generar datos de persona falsos
def generar_datos_persona():
    """
    Genera datos ficticios completos para una persona.
    
    Returns:
        dict: Diccionario con todos los datos de la persona.
    """
    # Generar nombre y apellidos separados
    nombre = eliminar_acentos(fake.first_name().upper())
    apellido1 = eliminar_acentos(fake.last_name().upper())
    apellido2 = eliminar_acentos(fake.last_name().upper())
    
    # Separar nombre en primer y segundo nombre (si existe)
    partes_nombre = nombre.split()
    primer_nombre = partes_nombre[0] if partes_nombre else ""
    segundo_nombre = " ".join(partes_nombre[1:]) if len(partes_nombre) > 1 else ""
    
    # Determinar género y edad
    genero = random.choice(["H", "M"])
    fecha_nac = fake.date_of_birth(minimum_age=18, maximum_age=90)

    # Usar la función para generar dirección
    direccion = generar_direccion_completa()
    
    seccion = str(random.randint(1, 9999)).zfill(4)
    
    # Vigencia como rango de años
    año_inicio = random.randint(2020, 2030)
    año_fin = año_inicio + 10
    vigencia = f"{año_inicio}-{año_fin}"
    
    # Año de registro con dos dígitos adicionales (mes)
    anio_registro = str(random.randint(2000, 2023))
    mes_registro = str(random.randint(1, 12)).zfill(2)  # Asegura que tenga 2 dígitos
    registro = f"{anio_registro} {mes_registro}"
    
    # Fecha de nacimiento en formato DD/MM/YYYY
    fecha_nacimiento = fecha_nac.strftime('%d/%m/%Y')
    
    # CURP y clave de elector
    curp = generar_curp().upper()
    clave_elector = generar_clave_elector().upper()
    
    return {
        "nombre": primer_nombre,
        "segundo_nombre": segundo_nombre,
        "apellido_paterno": apellido1,
        "apellido_materno": apellido2,
        "clave_elector": clave_elector,
        "curp": curp,
        "fecha_nacimiento": fecha_nacimiento,
        "sexo": genero,
        "seccion": seccion,
        "vigencia_inicio": str(año_inicio),
        "vigencia_fin": str(año_fin),
        "ano_registro": anio_registro,
        "mes_registro": mes_registro,
        "direccion1": direccion["direccion1"],
        "direccion2": direccion["direccion2"],
        "direccion3": direccion["direccion3"],
        "calle": direccion["calle"],
        "numero_ext": direccion["num_ext"],
        "numero_int": direccion["num_int"],
        "colonia": direccion["colonia"],
        "codigo_postal": direccion["cp"],
        "municipio": direccion["municipio"],
        "estado": direccion["estado"]
    }

In [10]:
# Cargar plantilla de INE y generar una falsa
def generar_ine_falsa(persona, ruta_destino):
    """
    Genera una imagen falsa de INE con los datos proporcionados.
    
    Args:
        persona (dict): Diccionario con los datos de la persona.
        ruta_destino (str): Ruta completa donde se guardará la imagen.
        
    Returns:
        bool: True si la imagen se generó correctamente, False en caso contrario.
    """
    try:
        # Cargar imagen de plantilla
        imagen = Image.open("plantilla_ine.png")
        
        # Convertir a RGB si está en modo RGBA
        if imagen.mode == 'RGBA':
            imagen = imagen.convert('RGB')
            
        draw = ImageDraw.Draw(imagen)
        
        # Configurar fuentes
        try:
            # Todas en negrita si es posible
            fuente_etiqueta = ImageFont.truetype("arial.ttf", size=12)  # Para etiquetas
            fuente_dato = ImageFont.truetype("arial.ttf", size=14)     # Para los datos
            fuente_grande = ImageFont.truetype("arial.ttf", size=16)   # Para nombre
        except IOError:
            fuente_etiqueta = ImageFont.load_default()
            fuente_dato = ImageFont.load_default()
            fuente_grande = ImageFont.load_default()
        
        # Ajustar posiciones para dejar espacio para la foto (mover todo a la derecha)
        offset_x = 270  # Aumentado para dejar más espacio para la foto
        offset_y = 170
        
        # NOMBRE
        draw.text((offset_x, offset_y), "NOMBRE", font=fuente_etiqueta, fill="black")
        nombre_completo = f"{persona['apellido_paterno']} {persona['apellido_materno']} {persona['nombre']}"
        if persona['segundo_nombre']:
            nombre_completo += f" {persona['segundo_nombre']}"
        draw.text((offset_x, offset_y + 15), persona["apellido_paterno"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 32), persona["apellido_materno"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 49), persona["nombre"], font=fuente_grande, fill="black")
        
        # SEXO - reposicionado para no encimarse
        draw.text((660, offset_y), "SEXO", font=fuente_etiqueta, fill="black")
        draw.text((696, offset_y - 1.5), persona["sexo"], font=fuente_dato, fill="black")
        
        # DOMICILIO - subido
        draw.text((offset_x, offset_y + 90), "DOMICILIO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 105), persona["direccion1"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 122), persona["direccion2"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 139), persona["direccion3"], font=fuente_grande, fill="black")
        
        # CLAVE DE ELECTOR (en la misma línea)
        draw.text((offset_x, offset_y + 180), "CLAVE DE ELECTOR", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 130, offset_y + 176), persona["clave_elector"], font=fuente_grande, fill="black")
        
        # AÑO DE REGISTRO (en la misma línea que CLAVE DE ELECTOR)
        draw.text((offset_x + 240, offset_y + 197), "AÑO DE REGISTRO", font=fuente_etiqueta, fill="black")
        registro = f"{persona['ano_registro']} {persona['mes_registro']}"
        draw.text((offset_x + 240, offset_y + 210), registro, font=fuente_grande, fill="black")
        
        # CURP (en la misma línea)
        draw.text((offset_x, offset_y + 197), "CURP", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 210), persona["curp"], font=fuente_grande, fill="black")
        
        # FECHA DE NACIMIENTO (en la misma línea)
        draw.text((offset_x, offset_y + 229), "FECHA DE NACIMIENTO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 242), persona["fecha_nacimiento"], font=fuente_grande, fill="black")
        
        # SECCIÓN (en la misma línea)
        draw.text((offset_x + 160, offset_y + 229), "SECCIÓN", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 160, offset_y + 242), persona["seccion"], font=fuente_grande, fill="black")
        
        # VIGENCIA (en la misma línea)
        draw.text((offset_x + 240, offset_y + 229), "VIGENCIA", font=fuente_etiqueta, fill="black")
        vigencia = f"{persona['vigencia_inicio']}-{persona['vigencia_fin']}"
        draw.text((offset_x + 240, offset_y + 242), vigencia, font=fuente_grande, fill="black")
        
        # Guardar la imagen
        imagen.save(ruta_destino)
        print(f"INE falsa generada: {ruta_destino}")
        return True
    except Exception as e:
        print(f"Error al generar INE falsa: {e}")
        return False

In [11]:
def guardar_metadata_jsonl(datos, ruta_archivo):
    """
    Guarda los datos en formato JSONL con el formato específico requerido.
    
    Args:
        datos (list): Lista de diccionarios con los datos a guardar.
        ruta_archivo (str): Ruta donde se guardará el archivo metadata.jsonl.
    """
    with open(ruta_archivo, 'w', encoding='utf-8') as archivo:
        for item in datos:
            # Crear el formato específico requerido
            gt_parse = {"gt_parse": item["datos"]}
            linea = {
                "file_name": item["archivo_imagen"],
                "ground_truth": json.dumps(gt_parse)
            }
            archivo.write(json.dumps(linea) + "\n")
    
    print(f"Archivo metadata.jsonl guardado en: {ruta_archivo}")

In [12]:
def exportar_datos_a_csv(personas, ruta_csv):
    """
    Exporta los datos de las personas a un archivo CSV.
    
    Args:
        personas (list): Lista de diccionarios con los datos de las personas.
        ruta_csv (str): Ruta donde se guardará el archivo CSV.
    """
    # Definir los encabezados del CSV según la estructura de los datos
    encabezados = [
        'Archivo_Imagen', 'Nombre', 'Segundo Nombre', 'Apellido Paterno', 'Apellido Materno',
        'Estado', 'Municipio', 'Colonia', 'Calle', 'Numero Ext.', 'Numero Int.',
        'Codigo Postal', 'CURP', 'Clave Elector', 'Fecha de Nacimiento',
        'Seccion', 'Vigencia Ano Inicio', 'Vigencia Ano Final', 'Ano Registro', 'Mes Registro'
    ]
    
    try:
        with open(ruta_csv, 'w', newline='', encoding='utf-8') as archivo_csv:
            writer = csv.DictWriter(archivo_csv, fieldnames=encabezados)
            writer.writeheader()
            
            for persona in personas:
                # Crear fila para CSV
                fila = {
                    'Archivo_Imagen': persona["archivo_imagen"],
                    'Nombre': persona["datos"]["nombre"],
                    'Segundo Nombre': persona["datos"]["segundo_nombre"],
                    'Apellido Paterno': persona["datos"]["apellido_paterno"],
                    'Apellido Materno': persona["datos"]["apellido_materno"],
                    'Estado': persona["datos"]["estado"],
                    'Municipio': persona["datos"]["municipio"],
                    'Colonia': persona["datos"]["colonia"],
                    'Calle': persona["datos"]["calle"],
                    'Numero Ext.': persona["datos"]["numero_ext"],
                    'Numero Int.': persona["datos"]["numero_int"],
                    'Codigo Postal': persona["datos"]["codigo_postal"],
                    'CURP': persona["datos"]["curp"],
                    'Clave Elector': persona["datos"]["clave_elector"],
                    'Fecha de Nacimiento': persona["datos"]["fecha_nacimiento"],
                    'Seccion': persona["datos"]["seccion"],
                    'Vigencia Ano Inicio': persona["datos"]["vigencia_inicio"],
                    'Vigencia Ano Final': persona["datos"]["vigencia_fin"],
                    'Ano Registro': persona["datos"]["ano_registro"],
                    'Mes Registro': persona["datos"]["mes_registro"]
                }
                writer.writerow(fila)
                
        print(f"Archivo CSV generado exitosamente en: {ruta_csv}")
        return True
    except Exception as e:
        print(f"Error al generar el archivo CSV: {e}")
        return False

In [13]:
def generar_dataset(cantidad, prop_train=0.7, prop_val=0.2, prop_test=0.1):
    """
    Genera un dataset completo con imágenes y metadatos, dividiéndolo en conjuntos
    de entrenamiento, validación y prueba.
    
    Args:
        cantidad (int): Número total de imágenes a generar.
        prop_train (float): Proporción para entrenamiento (0.0-1.0).
        prop_val (float): Proporción para validación (0.0-1.0).
        prop_test (float): Proporción para prueba (0.0-1.0).
    
    Returns:
        tuple: Conteos de imágenes en cada conjunto (train, val, test).
    """
    # Verificar que las proporciones sumen 1.0
    if abs((prop_train + prop_val + prop_test) - 1.0) > 0.001:
        raise ValueError(f"Las proporciones deben sumar 1.0, pero suman {prop_train + prop_val + prop_test}")
    
    # Crear la estructura de carpetas
    crear_estructura_carpetas()
    
    # Calcular la cantidad de imágenes para cada conjunto
    n_train = int(cantidad * prop_train)
    n_val = int(cantidad * prop_val)
    n_test = cantidad - n_train - n_val  # Para asegurar que sumen exactamente 'cantidad'
    
    print(f"Se generarán:")
    print(f"- {n_train} imágenes para entrenamiento")
    print(f"- {n_val} imágenes para validación")
    print(f"- {n_test} imágenes para prueba")
    
    # Listas para almacenar los datos de cada conjunto
    personas_train = []
    personas_val = []
    personas_test = []
    todas_personas = []  # Para el CSV general
    
    # Generar las imágenes y distribuirlas en conjuntos
    for i in tqdm(range(cantidad), desc="Generando dataset"):
        # Generar nombre de archivo
        nombre_archivo = f"ine_falsa_{i+1:03d}.jpg"
        
        # Generar datos de persona
        datos_persona = generar_datos_persona()
        
        # Crear objeto con datos e imagen correspondiente
        persona = {
            "archivo_imagen": nombre_archivo,
            "datos": datos_persona
        }
        
        todas_personas.append(persona)  # Agregar a la lista general
        
        # Determinar a qué conjunto pertenece esta imagen
        if i < n_train:
            # Conjunto de entrenamiento
            ruta_destino = os.path.join(TRAIN_DIR, nombre_archivo)
            personas_train.append(persona)
            conjunto = "entrenamiento"
        elif i < n_train + n_val:
            # Conjunto de validación
            ruta_destino = os.path.join(VAL_DIR, nombre_archivo)
            personas_val.append(persona)
            conjunto = "validación"
        else:
            # Conjunto de prueba
            ruta_destino = os.path.join(TEST_DIR, nombre_archivo)
            personas_test.append(persona)
            conjunto = "prueba"
        
        # Generar la imagen en la carpeta correspondiente
        if generar_ine_falsa(datos_persona, ruta_destino):
            # También guardar una copia en DATA_DIR
            ruta_data = os.path.join(DATA_DIR, nombre_archivo)
            shutil.copy2(ruta_destino, ruta_data)
        else:
            print(f"Error al generar imagen para conjunto de {conjunto}: {nombre_archivo}")
    
    # Generar los archivos metadata.jsonl para cada conjunto
    guardar_metadata_jsonl(personas_train, os.path.join(TRAIN_DIR, "metadata.jsonl"))
    guardar_metadata_jsonl(personas_val, os.path.join(VAL_DIR, "metadata.jsonl"))
    guardar_metadata_jsonl(personas_test, os.path.join(TEST_DIR, "metadata.jsonl"))
    
    # Generar el CSV con todos los datos
    ruta_csv = os.path.join(BASE_DIR, "datos_ines.csv")
    exportar_datos_a_csv(todas_personas, ruta_csv)
    
    print(f"\nDataset generado con éxito:")
    print(f"- {len(personas_train)} imágenes en conjunto de entrenamiento")
    print(f"- {len(personas_val)} imágenes en conjunto de validación")
    print(f"- {len(personas_test)} imágenes en conjunto de prueba")
    print(f"- Total: {len(todas_personas)} imágenes")
    print(f"\nArchivo CSV guardado en: {ruta_csv}")
    print(f"Todos los archivos metadata.jsonl han sido generados en sus respectivas carpetas")
    
    return len(personas_train), len(personas_val), len(personas_test)

In [14]:
if __name__ == "__main__":
    # Número de INEs a generar
    num_ines = 300
    
    # Proporciones para cada conjunto (deben sumar 1.0)
    prop_train = 0.7  # 70% para entrenamiento
    prop_val = 0.2    # 20% para validación
    prop_test = 0.1   # 10% para prueba
    
    # Generar el dataset
    generar_dataset(num_ines, prop_train, prop_val, prop_test)

Estructura de carpetas creada correctamente.
Se generarán:
- 210 imágenes para entrenamiento
- 60 imágenes para validación
- 30 imágenes para prueba


Generando dataset:   0%|          | 1/300 [00:00<00:29,  9.99it/s]

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\ine_falsa_001.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\ine_falsa_002.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\ine_falsa_003.jpg


Generando dataset:   1%|▏         | 4/300 [00:00<00:16, 17.76it/s]

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\ine_falsa_004.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\ine_falsa_005.jpg


...

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_289.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_290.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_291.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_292.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_293.jpg


Generando dataset:  99%|█████████▊| 296/300 [00:13<00:00, 21.63it/s]

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_294.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_295.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_296.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_297.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_298.jpg


Generando dataset: 100%|██████████| 300/300 [00:14<00:00, 21.35it/s]

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_299.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\ine_falsa_300.jpg
Archivo metadata.jsonl guardado en: C:\Users\Adrian\Desktop\INEs Dataset\sets\train\metadata.jsonl
Archivo metadata.jsonl guardado en: C:\Users\Adrian\Desktop\INEs Dataset\sets\validation\metadata.jsonl
Archivo metadata.jsonl guardado en: C:\Users\Adrian\Desktop\INEs Dataset\sets\test\metadata.jsonl
Archivo CSV generado exitosamente en: C:\Users\Adrian\Desktop\INEs Dataset\datos_ines.csv

Dataset generado con éxito:
- 210 imágenes en conjunto de entrenamiento
- 60 imágenes en conjunto de validación
- 30 imágenes en conjunto de prueba
- Total: 300 imágenes

Archivo CSV guardado en: C:\Users\Adrian\Desktop\INEs Dataset\datos_ines.csv
Todos los archivos metadata.jsonl han sido generados en sus respectivas carpetas
